In [1]:
!pip install pysnmp

In [2]:
import pysnmp

In [7]:
from pysnmp.hlapi import *

No SNMP response received before timeout


In [8]:
from pysnmp.hlapi import *

def snmp_walk(ip, oid):
    for (errorIndication, errorStatus, errorIndex, varBinds) in nextCmd(
        SnmpEngine(),
        CommunityData('public', mpModel=0),
        UdpTransportTarget((ip, 161)),
        ContextData(),
        ObjectType(ObjectIdentity(oid)),
        lexicographicMode=False
    ):
        if errorIndication:
            print(errorIndication)
            break
        elif errorStatus:
            print('%s at %s' % (errorStatus.prettyPrint(),
                                errorIndex and varBinds[int(errorIndex) - 1][0] or '?'))
            break
        else:
            for varBind in varBinds:
                print(f'{varBind[0]} = {varBind[1]}')

In [9]:
# Printer IP address (replace with your printer's IP)
printer_ip = '192.168.1.2'

In [10]:
# OID for the printer MIB (replace if necessary)
base_oid = '1.3.6.1.2.1.43'

In [11]:
# Perform the SNMP walk
snmp_walk(printer_ip, base_oid)

No SNMP response received before timeout


In [ ]:
from pysnmp.hlapi import *
import csv
from datetime import datetime

# List of printer IP addresses (replace with your printers' IPs)
printer_ips = ['192.168.1.2', '192.168.1.3', '192.168.1.4']

# Replace these with the OIDs you discovered from the SNMP walk
oids = {
    'black_white': 'YOUR_BLACK_WHITE_OID',
    'color': 'YOUR_COLOR_OID',
    'total': 'YOUR_TOTAL_OID'
}

def get_snmp_data(ip, oid):
    iterator = getCmd(
        SnmpEngine(),
        CommunityData('public', mpModel=0),
        UdpTransportTarget((ip, 161)),
        ContextData(),
        ObjectType(ObjectIdentity(oid))
    )
    errorIndication, errorStatus, errorIndex, varBinds = next(iterator)
    
    if errorIndication:
        print(f"Error with IP {ip}: {errorIndication}")
        return None
    elif errorStatus:
        print('%s at %s' % (errorStatus.prettyPrint(),
                            errorIndex and varBinds[int(errorIndex) - 1][0] or '?'))
        return None
    else:
        for varBind in varBinds:
            return int(varBind[1])
    return None

# Collect data
data = []
for ip in printer_ips:
    row = {'ip': ip}
    for key, oid in oids.items():
        row[key] = get_snmp_data(ip, oid)
    data.append(row)

# Save data to CSV
filename = f'printer_data_{datetime.now().strftime("%Y-%m-%d")}.csv'
with open(filename, mode='w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=['ip', 'black_white', 'color', 'total'])
    writer.writeheader()
    writer.writerows(data)

print(f'Data saved to {filename}')
